In [1]:
#importing some libraries

import numpy as np
import urllib.request


In [25]:
# initialization of the similarity matrix
A = input("Enter the first sequence: ")
B = input("Enter the second sequence: ")


A = [i for i in A]
B = [i for i in B]

#to print the similarity matrix

G = np.zeros((len(A),len(B)))                                   
#G = [[0 for i in range(len(A))] for j in range(len(B))]


print(A)
print(B)
print(G[3][4])

Enter the first sequence: SGAKSMWTEHKSPDGRTYYYNTETKQSTWEKPDD
Enter the second sequence: LLSKCPWKEYKSDSGKPYYYNSQTKESRWAKPKE
['S', 'G', 'A', 'K', 'S', 'M', 'W', 'T', 'E', 'H', 'K', 'S', 'P', 'D', 'G', 'R', 'T', 'Y', 'Y', 'Y', 'N', 'T', 'E', 'T', 'K', 'Q', 'S', 'T', 'W', 'E', 'K', 'P', 'D', 'D']
['L', 'L', 'S', 'K', 'C', 'P', 'W', 'K', 'E', 'Y', 'K', 'S', 'D', 'S', 'G', 'K', 'P', 'Y', 'Y', 'Y', 'N', 'S', 'Q', 'T', 'K', 'E', 'S', 'R', 'W', 'A', 'K', 'P', 'K', 'E']
0.0


In [33]:
# constructing subsitution matrix
# I used the link below to fetch the matrix after uploading the matrices to the git repository
blosum62_url = 'https://raw.githubusercontent.com/moomen91/BioInformatics/master/blosum62.txt'
blosum80_url = 'https://raw.githubusercontent.com/moomen91/BioInformatics/master/blosum80.txt'
pam60_url = 'https://raw.githubusercontent.com/moomen91/BioInformatics/master/pam60.txt'
pam120_url = 'https://raw.githubusercontent.com/moomen91/BioInformatics/master/pam120.txt'


def get_substitution_matrix(url):
    

    f = urllib.request.urlopen(url) 

    file = f.readlines()

    substitution_matrix = list()
    
    for line in file:
        line = line.split()
        line = [i.decode('utf-8') for i in line]
        if('#' not in line):
            
            substitution_matrix.append(line)

    #making multidimentional dictionary to map all the characters

    matrix_dict = {}

    for i in range(len(substitution_matrix)-1):
    
        matrix_dict[substitution_matrix[i][0]] = {}
    
        for j in range(len(substitution_matrix[0])-1):
        
            matrix_dict[substitution_matrix[i][0]][substitution_matrix[0][j]] = substitution_matrix[i][j + 1]
            
    return matrix_dict

#creating all the matrices by calling the fuction of get_substitution_matrix

blosum62 = get_substitution_matrix(blosum62_url)
blosum80 = get_substitution_matrix(blosum80_url)
pam60 = get_substitution_matrix(pam60_url)
pam120 = get_substitution_matrix(pam120_url)

print(blosum62['A']['N'])

-2


In [34]:
#filling in the matrix
# the pseudo code for the following implementation can be found on the following link
# https://en.wikipedia.org/wiki/Needleman%E2%80%93Wunsch_algorithm#Scoring_systems

def createScoringMatrix(seq_A, seq_B, F, S, penalty_gap):
    
    for i in range(len(seq_A)-1):
        
        F[i][0] = penalty_gap*i
        
    for j in range(len(seq_B)-1):
        
        F[0][j] = penalty_gap*j
        
    for i in range(len(seq_A)-1):
        
        for j in range(len(seq_B)-1):
            
            match = F[i-1][j-1] + int(S[seq_A[i]][seq_B[j]])
            delete = F[i-1][j] + penalty_gap
            insert = F[i][j-1] + penalty_gap
            
            F[i][j] = max(match, delete, insert)
            
    return F

X = createScoringMatrix(seq_A = A, seq_B = B, F = G, S = blosum_dict, penalty_gap = -12)
print(np.matrix(X))

[[-2. -2.  4. ... -1.  0.  0.]
 [-4. -6. -2. ... -2. -3.  0.]
 [-1. -5. -5. ... -2. -3.  0.]
 ...
 [-3. -5. -6. ... 96. 84.  0.]
 [-4. -7. -5. ... 84. 95.  0.]
 [ 0.  0.  0. ...  0.  0.  0.]]


In [36]:
# to compute an allignment using the algorithm

def compute_alignment(seq_A, seq_B, F, S, penalty_gap):
    
    alignment_A = ""
    alignment_B = ""
    i = len(A) - 1
    j = len(B) - 1

    while(i >= 0 or j >= 0):
    
        if(i >= 0 and j >= 0 and (F[i][j] == F[i - 1][j - 1] + int(blosum_dict[A[i]][B[j]]))):
        
            alignment_A += seq_A[i]
            alignment_B += seq_B[j]
            i -= 1
            j -= 1
        elif( i >= 0 and j >= 0 and F[i][j] == F[i-1][j] + (penalty_gap)):
            alignment_A += seq_A[i]
            alignment_B += "-"
            
            i -= 1
            j -= 1
        else:
        
            alignment_A += "-"
            alignment_B += seq_B[j]
            
            i -= 1
            j -= 1
            
    print(alignment_A[::-1])
    print()
    print(alignment_B[::-1])
    

compute_alignment(seq_A = A, seq_B = B, F = X, S = blosum_dict, penalty_gap = -12)
    

SGAKSMWTEHKSPDGRTYYYNTETKQSTWEKPD-

LLSKCPWKEYKSDSGKPYYYNSQTKESRWAKPKE
